# Импортируем необходимые библиотеки
# Разделение данных на train, validation и test с распределением 60/20/20

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error
import xgboost as xgb
import numpy as np

# Загрузка данных 
df = pd.read_csv('data/jamb_exam_results.csv') 

# Преобразование названий колонок к нижнему регистру и замена пробелов на подчеркивания
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Удаление столбца student_id
df = df.drop(columns=['student_id'])

# Заполнение пропущенных значений нулями
df = df.fillna(0)


df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)  # 0.25 * 0.8 = 0.2

# Выделение целевой переменной и признаков
y_train = df_train.jamb_score.values
y_val = df_val.jamb_score.values
y_test = df_test.jamb_score.values

df_train = df_train.drop(columns=['jamb_score'])
df_val = df_val.drop(columns=['jamb_score'])
df_test = df_test.drop(columns=['jamb_score'])

# Преобразование датафреймов в матрицы
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(df_train.to_dict(orient='records'))
X_val = dv.transform(df_val.to_dict(orient='records'))
X_test = dv.transform(df_test.to_dict(orient='records'))

# Вопрос 1: Обучение дерева решений

In [19]:
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

# Получение признака, используемого для разбиения
feature = dv.get_feature_names_out()[dt.tree_.feature[0]]
print("Вопрос 1: Признак для разбиения данных:", feature)

Вопрос 1: Признак для разбиения данных: study_hours_per_week


# Вопрос 2: Обучение случайного леса

In [20]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
rmse_val = root_mean_squared_error(y_val, y_pred)
print("Вопрос 2: RMSE случайного леса на валидационных данных:", round(rmse_val, 2))

Вопрос 2: RMSE случайного леса на валидационных данных: 42.14


# Вопрос 3: Эксперимент с n_estimators

In [21]:
rmse_values = []
for n in range(10, 201, 10):
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    rmse_values.append(rmse)

# Поиск значения n_estimators, после которого RMSE перестает улучшаться
best_n_estimators = 10 + (10 * np.argmin(np.diff(rmse_values) < 0.001))
print("Вопрос 3: n_estimators, после которого RMSE перестает улучшаться:", best_n_estimators)

Вопрос 3: n_estimators, после которого RMSE перестает улучшаться: 90


# Вопрос 4: Поиск лучшего значения max_depth

In [22]:

best_rmse = float('inf')
best_depth = None
for depth in [10, 15, 20, 25]:
    rmse_list = []
    for n in range(10, 201, 10):
        rf = RandomForestRegressor(n_estimators=n, max_depth=depth, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        rmse_list.append(rmse)
    
    avg_rmse = np.mean(rmse_list)
    if avg_rmse < best_rmse:
        best_rmse = avg_rmse
        best_depth = depth

print("Вопрос 4: Лучшее значение max_depth по среднему RMSE:", best_depth)

Вопрос 4: Лучшее значение max_depth по среднему RMSE: 10


# Вопрос 5: Определение важности признаков

In [23]:
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

# Получение важности признаков
feature_importances = rf.feature_importances_
important_feature = dv.get_feature_names_out()[np.argmax(feature_importances)]
print("Вопрос 5: Самый важный признак:", important_feature)

Вопрос 5: Самый важный признак: study_hours_per_week


# Вопрос 6: Обучение модели XGBoost с разными значениями eta

In [24]:
# Настройки для XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
watchlist = [(dtrain, 'train'), (dval, 'eval')]

# Модель с eta=0.3
xgb_params_03 = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'seed': 1,
    'verbosity': 1
}
model_03 = xgb.train(xgb_params_03, dtrain, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)

# Модель с eta=0.1
xgb_params_01 = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'seed': 1,
    'verbosity': 1
}
model_01 = xgb.train(xgb_params_01, dtrain, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)

# Сравнение RMSE моделей
rmse_03 = model_03.best_score
rmse_01 = model_01.best_score

if rmse_03 < rmse_01:
    best_eta = 0.3
elif rmse_01 < rmse_03:
    best_eta = 0.1
else:
    best_eta = 'Both give equal value'

print("Вопрос 6: Лучшее значение eta:", best_eta)

Вопрос 6: Лучшее значение eta: 0.1
